In [1]:
import pandas as pd

merged_data = pd.read_csv('data_simple.csv', encoding = "ISO-8859-1", low_memory=False, index_col=0)

In [3]:
merged_data.describe()

,ADD_DATE,ARREST_DATE,BAR_ADMISSION,CHARGE_CLASS,DFDN_SEQ_NBR,DOB,POLICE_RPT_DATE,POLICE_RPT_DAYS,SADA_DOB,SCREENING_DAYS,SCREENING_DISP_CODE,SCREENING_DISP_DATE,SYS_NBR
count,2.802900e+05,2.802900e+05,2.494070e+05,275692.000000,280294.000000,2.802940e+05,2.802940e+05,280294.000000,2.438620e+05,280294.000000,244779.00000,2.802940e+05,2.802940e+05
mean,1.979663e+07,1.935621e+07,1.989125e+07,3.275510,1.097765,1.934283e+07,1.712800e+07,32.258093,1.960577e+07,15.128747,238.36247,1.742101e+07,9.417777e+07
std,1.689978e+06,3.360846e+06,6.791726e+04,0.872969,0.483895,2.580123e+06,6.946953e+06,66.161661,8.182868e+04,34.381894,10.18528,6.628696e+06,3.470239e+06
min,0.000000e+00,0.000000e+00,1.905061e+07,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,1.901050e+07,0.000000,17.00000,0.000000e+00,8.400002e+07
25%,1.991063e+07,1.991031e+07,1.987101e+07,3.000000,1.000000,1.961013e+07,1.990082e+07,3.000000,1.960000e+07,0.000000,230.00000,1.990061e+07,9.110910e+07
50%,1.994090e+07,1.994053e+07,1.990100e+07,3.000000,1.000000,1.970103e+07,1.994052e+07,13.000000,1.963000e+07,5.000000,240.00000,1.994022e+07,9.417507e+07
75%,1.997071e+07,1.997052e+07,1.992102e+07,4.000000,1.000000,1.977082e+07,1.997062e+07,34.000000,1.966000e+07,16.000000,240.00000,1.997052e+07,9.724202e+07
max,1.999112e+07,1.999112e+07,1.999042e+07,8.000000,27.000000,1.999090e+07,1.999112e+07,999.000000,1.974000e+07,997.000000,560.00000,1.999112e+07,9.930916e+07


In [4]:
merged_data.columns

Index(['ADA_CODE', 'ADD_DATE', 'ARREST_CREDIT_CODE', 'ARREST_DATE',
       'BAR_ADMISSION', 'BOFI_NBR', 'CHARGE_CLASS', 'CHARGE_TYPE',
       'CRIMINAL_FLAG', 'DFDN_SEQ_NBR', 'DOB', 'FBI_NBR',
       'FINAL_DETENTION_FLAG', 'HABITUAL_OFFENDER_FLAG',
       'INITIAL_DETENTION_FLAG', 'JUVENILE_FLAG', 'LEAD_CHARGE_CODE', 'PARTY',
       'POLICE_RPT_DATE', 'POLICE_RPT_DAYS', 'RACE', 'SADA_DOB', 'SADA_RACE',
       'SADA_SEX', 'SCREENING_DAYS', 'SCREENING_DISP_CODE',
       'SCREENING_DISP_DATE', 'SEX', 'SYS_NBR'],
      dtype='object')

In [105]:
categorical = ['ARREST_CREDIT_CODE', 'CHARGE_CLASS', 'CHARGE_TYPE', 'LEAD_CHARGE_CODE',\
               'PARTY', 'RACE', 'SADA_RACE']

In [106]:
#create a summary of each one and store it in a list or dictionary I guess

dict_counts = {}

for columnname in categorical:
    dict_counts[columnname] = merged_data[columnname].value_counts()

In [109]:
len(merged_data['ARREST_CREDIT_CODE'].unique())

103

In [110]:
merged_data[categorical].isnull().sum()

ARREST_CREDIT_CODE    10878
CHARGE_CLASS           4602
CHARGE_TYPE             581
LEAD_CHARGE_CODE        552
PARTY                 56195
RACE                   4821
SADA_RACE             38901
dtype: int64

In [111]:
merged_cat_filled = merged_data[categorical].fillna('NA')

In [49]:
merged_cat_filled.isnull().sum()

CHARGE_CLASS        0
CHARGE_TYPE         0
CRIMINAL_FLAG       0
LEAD_CHARGE_CODE    0
PARTY               0
RACE                0
SADA_RACE           0
dtype: int64

In [65]:
invalid_charge_codes = ['40:(979)296', 
               '14:(24)30(',
               '14:(24)67(',
               '5:606',
               '13:34',
               'F5:257',
               '14:(26)67(',
               '40:(979)1967',
               '4:664']

In [84]:
merged_cat_filled[merged_cat_filled['LEAD_CHARGE_CODE'].isin(invalid_charge_codes) == True]

,CHARGE_CLASS,CHARGE_TYPE,CRIMINAL_FLAG,LEAD_CHARGE_CODE,PARTY,RACE,SADA_RACE
UNIQUE_ID,,,,,,,
5207,6,NA,N,5:606,D,NA,W
163070,NA,NA,Y,40:(979)1967,D,N,W
196149,4,AR,Y,F5:257,R,NA,W
202726,NA,AR,Y,13:34,D,N,B
203134,NA,AR,Y,4:664,R,O,W
221631,NA,AR,Y,40:(979)296,D,N,B
241333,NA,AR,Y,14:(24)30(,D,W,W
241335,NA,AR,Y,14:(24)30(,D,W,W
242808,2,AR,Y,40:(979)296,D,W,T


In [74]:
merged_cat_filled['LEAD_CHARGE_CODE'].replace(invalid_charge_codes, 'NA', inplace=True)

In [62]:
merged_cat_filled['RACE'].replace('N', 'B', inplace=True)
#the codebook says that the value A should not be accepted, but this refers to "asian" elsewhere
#so I'm hesitant to throw out the A's
merged_cat_filled['RACE'].replace('O','A', inplace=True)

In [112]:
merged_cat_filled['ARREST_CREDIT_CODE'].replace('00', 'NA', inplace=True)

In [87]:
categorical

['CHARGE_CLASS',
 'CHARGE_TYPE',
 'LEAD_CHARGE_CODE',
 'PARTY',
 'RACE',
 'SADA_RACE']

In [90]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [114]:
'''
create features using one hot encoding
'''

# first convert into integer values. one hot enconding only takes int input

l_enc = LabelEncoder()

col1 = l_enc.fit_transform(merged_cat_filled['CHARGE_CLASS'].astype(str))
col2 = l_enc.fit_transform(merged_cat_filled['CHARGE_TYPE'])
col3 = l_enc.fit_transform(merged_cat_filled['LEAD_CHARGE_CODE'])
col4 = l_enc.fit_transform(merged_cat_filled['PARTY'])
col5 = l_enc.fit_transform(merged_cat_filled['RACE'])
col6 = l_enc.fit_transform(merged_cat_filled['SADA_RACE'])
col7 = l_enc.fit_transform(merged_cat_filled['ARREST_CREDIT_CODE'])

X = np.column_stack((col1,col2,col3,col4,col5,col6,col7))

#one hot encoding
enc = OneHotEncoder()
X_enc = enc.fit_transform(X)

In [ ]:
#may also import table Code to store interpretation of each category
#ARCDCD in Code table: Arrest credit code
#CGCD table: Lead charge code
#CSCLC in Code table: Charge class